In [1]:
import copy
import numpy as np
import os

import pickle

import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import time

from torch.autograd import Variable

# from extractor import get_features
# from extractor
os.listdir()

data_dir = 'data'
train_np_dir = 'train_np'
test_np_dir = 'test_np'
trained_weights_dir = 'trained_weights'

In [2]:
# Load existing clasess
pkl_file = open('classes_dict.pickle', 'rb')

classes_dict= pickle.load(pkl_file)

pkl_file.close()

print(classes_dict['PlayingGuitar'], len(classes_dict))

62 101


In [3]:
hidden_size = 64
input_size = 512
num_layers = 2
batch_size = 300
num_epochs = 100
learning_rate = 0.01
num_classes = len(classes_dict)
sequence_length = 50
# label_str_to_int = {'ApplyEyeMakeup': 0, 'Archery': 1, 'ApplyLipstick': 2}

### Define models

In [6]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

rnn = RNN(input_size=512, hidden_size=hidden_size,
          num_layers=num_layers, num_classes=num_classes).cuda()
rnn


RNN (
  (lstm): LSTM(512, 64, num_layers=2, batch_first=True)
  (fc): Linear (64 -> 101)
)

In [7]:
# RNN Model (Many-to-One)
class GRU_rnn(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU_rnn, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

gru = GRU_rnn(input_size=512, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes).cuda()
gru


GRU_rnn (
  (lstm): GRU(512, 64, num_layers=2, batch_first=True)
  (fc): Linear (64 -> 101)
)

 _____________________________________________________

In [8]:
use_gpu = torch.cuda.is_available()
print ("GPU is available: ", use_gpu)

GPU is available:  True


In [9]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate, weight_decay=0.1)

In [10]:
def exp_lr_scheduler(optimizer, epoch, init_lr=0.01, lr_decay_epoch=14):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.5**(epoch // lr_decay_epoch))
#     lr = init_lr #* 1 / (1 + )
    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer


In [11]:
def copute_dataset_size(path=''):
    data_classes = [i for i in os.listdir(path) if not i.startswith('.')]
    num_entries = 0

    for folder in data_classes:
        current_dir = path + '/' + folder + '/'
        num_entries += len(os.listdir(current_dir))
        
    return num_entries

train_size = copute_dataset_size(path=data_dir + '/' + train_np_dir)
test_size = copute_dataset_size(path=data_dir + '/' + test_np_dir)

def check_size(address, sequence_size=50):
    x = np.load(address)
    print (x.shape)
    if x.shape[0] < sequence_size:
        return np.concatenate((x, np.zeros((sequence_size - x.shape[0], x.shape[1]))), axis=0)
    return x

# Data Loader (Input Pipeline)
def get_loader(path=''):
    """
    Function reads .npy files from path.
    Returns:
        dataloader, data classes (list), size of input object [n_sequence, n_features], lenght_of_dataset
    """
    inputs = []
    targets = []
    data_classes = [i for i in os.listdir(path) if not i.startswith('.')]

    for folder in data_classes:
        current_dir = path + '/' + folder + '/'
        files = os.listdir(current_dir)
        #test_f = np.load(current_dir + files[0])[:,:30,:]
        
#         print(test_f.shape)
        temp = [torch.Tensor(np.load(current_dir +  f).reshape((sequence_length, input_size))) for f in files] 
                         # Transform to torch tensors
        
        targets += ([torch.LongTensor([classes_dict[folder]])] * len(temp))
        inputs += temp

    tensor_x = torch.stack(inputs)
    tensor_y = torch.stack(targets)
    my_dataset = torch.utils.data.TensorDataset(tensor_x, tensor_y)  # Create your datset
    my_dataloader = torch.utils.data.DataLoader(my_dataset, batch_size=batch_size, shuffle=True)  # Create your dataloader

    return (my_dataloader, data_classes)

dset_loaders = {x: get_loader(data_dir + '/' + x)[0] for x in [train_np_dir, test_np_dir]}

In [12]:
# os.listdir(data_dir + '/' + test_np_dir)

In [13]:
dset_sizes = {train_np_dir: train_size, test_np_dir: test_size}
dset_sizes

{'test_np': 3783, 'train_np': 9522}

In [ ]:
def train_model(model, criterion, optimizer, lr_scheduler, path, num_epochs=200, model_name='lstm'):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in [train_np_dir, test_np_dir]:
            if phase == train_np_dir:
                optimizer = lr_scheduler(optimizer, epoch)
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            counter = 0
            # Iterate over data.
            for data in dset_loaders[phase]:
                # get the inputs

                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
#                     print(inputs)
                    inputs = Variable(inputs.view(-1, sequence_length, input_size).cuda())
                    labels = Variable(labels.view(-1).cuda())                        
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                    
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
#                 print (inputs.size())
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
#                 print (labels, outputs)

                # backward + optimize only if in training phase
                if phase == train_np_dir:
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

#             print(preds, labels.data, '\n========')
            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects / dset_sizes[phase]

            print('running corrects: ', running_corrects)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == test_np_dir and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model)
        # saving weights
        torch.save(model, data_dir + '/' + trained_weights_dir + "/" + model_name + '_' + str(epoch) + ".pt")

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return best_model


In [ ]:
model_lstm = train_model(rnn, criterion, optimizer, exp_lr_scheduler, '', num_epochs=num_epochs, model_name='lstm')

Epoch 0/99
----------
LR is set to 0.01
Variable containing:
 89
 36
 40
 58
  8
 17
  7
 39
 95
 53
 51
 96
 58
 32
 33
 13
  8
 28
 58
 26
  8
 38
 89
 80
 91
 14
 84
  6
 28
 77
 21
 68
 77
 52
  7
 89
 83
 87
 99
 85
 26
 23
 79
 80
 16
 21
 13
 38
 91
 84
 45
  2
 25
 38
 33
 63
 81
 16
 36
 88
  5
 11
 59
  0
 29
 61
 86
 89
 81
 66
 30
 33
 34
  1
 96
  5
 24
 14
 31
  8
 29
 39
 61
 21
 44
 61
 97
 20
 72
 38
 46
 16
 17
  9
 94
 12
 34
 12
 98
 57
 82
 93
 42
 30
 65
 56
 85
 63
 33
 78
 64
 14
 53
 83
 55
 72
 46
 19
 56
 59
 94
 75
 38
 17
 68
 50
 91
 53
 40
 90
 51
 78
 99
 95
 16
 30
 84
 11
 58
 59
 99
 46
 19
 92
 90
 66
 94
  2
 76
  1
 78
 56
 15
 44
 56
 21
 79
 55
 48
 37
 16
 99
  3
 31
 16
 29
 16
 83
 34
 44
 24
 28
 26
 42
 26
 82
 83
 54
 93
 55
  9
 11
 58
 26
 56
 59
 58
 17
 21
 50
 15
 27
 43
 38
 64
 74
 45
  7
 57
  2
 23
 62
 62
 36
 69
  7
 60
 52
 26
 72
 54
 36
 14
 95
 26
 10
 38
 70
 74
 19
 28
 64
 90
 39
 68
 76
 12
 95
 36
 94
 55
 94
 32
 36
 86


Variable containing:
  83
  30
   5
  34
  92
   8
  73
  56
  42
  75
  38
  60
  59
   4
  25
  79
  91
  21
   4
  89
   4
  80
  60
  56
  90
  65
  42
  92
  84
  14
  11
  91
  48
  50
   8
  52
  72
  50
  72
  60
  56
  64
  56
  47
  72
  61
   9
  86
  97
  47
  64
  33
  83
  44
  67
  52
   8
  92
  61
  50
  92
  26
  69
  78
  85
   7
   2
  70
  17
  16
  84
   3
  53
  83
  82
  75
  70
  42
  81
  74
  88
   9
  59
  19
  17
  18
  57
  70
  38
  61
  38
  61
  94
  52
  73
   4
  38
  47
  84
  30
  27
  77
  78
  40
  72
  76
  73
  64
  91
  84
  53
  72
   5
  92
  54
  91
  44
  93
  11
  88
  33
   9
  55
  98
  11
  99
  45
  37
  85
  17
  84
  26
  98
  86
  23
  88
  83
  92
  83
  15
  87
   4
  97
  17
 100
   5
  34
  29
  51
  65
   1
  55
  61
  51
   2
  91
  80
  86
  28
  33
  81
  85
  48
  49
  53
  66
  74
  58
  80
  89
  62
  24
  46
  88
  91
  35
  41
  11
  42
  31
  25
  98
  31
  60
  32
 100
  82
  77
  56
  72
  33
  35
  15
   3
   5
  4


Variable containing:
  48
  56
  61
  78
  33
  85
  46
  25
   2
  74
  83
  67
  21
  69
  99
  65
  80
  74
  35
  56
  59
  64
  68
  79
  22
  74
  20
  75
  42
  56
  13
  70
  53
  84
  68
  87
  94
  43
  53
  50
  34
  91
  88
  12
 100
  32
  28
  99
  64
  12
  25
  38
  11
  60
  49
  52
  23
  30
  71
  10
  57
  78
  87
   8
  31
  90
  48
  30
  58
  15
  35
  27
  38
  63
  58
  29
  61
   8
   1
  47
  70
  41
  94
  17
   3
  54
   2
  53
  24
  64
  95
  59
  46
  28
  73
  42
  56
  56
  87
  99
  97
  79
   3
   1
  52
  88
  44
 100
  27
  74
  26
  51
  57
  11
   5
  97
   2
   0
  26
  43
   9
   8
  65
  45
  57
  56
  25
  39
  50
  66
  63
   3
  34
  91
  98
  18
  85
  18
   5
  39
  86
  62
  91
  97
   0
   8
  21
  57
  83
  16
  24
  73
  77
  62
  81
  91
  28
  57
  97
  46
  16
  85
   5
  52
  62
  63
  66
  72
  23
  46
  70
  35
  64
  73
  68
  38
  17
  84
  82
  55
  39
  73
  21
  27
  52
  49
  48
  56
  59
  11
   1
  31
  46
  82
   8
  9


Variable containing:
  42
  50
  75
  91
   1
  42
  24
  50
  72
  60
  31
  57
  50
  91
  25
   0
  76
  82
  34
  48
   5
  70
  61
   1
   0
   6
  96
  76
  29
  93
  34
  47
   1
  53
  50
  80
  47
  59
  50
  70
   5
  26
  22
   7
  36
  84
  65
  17
  46
  41
  23
  40
  47
  16
  71
  84
  24
  69
  73
  85
  76
  42
  94
  66
  47
  93
  21
   2
  91
  56
  83
  50
  90
  26
   4
  28
   3
  19
  49
  38
  50
  35
 100
  53
  56
  48
  92
  83
  10
   8
  31
  21
  37
  84
  29
  42
  55
  79
  30
  22
  75
  21
  78
  74
  97
  17
  98
  17
  15
  84
  54
  38
  51
  76
  52
  61
  23
  34
   6
   2
  25
  96
  15
   2
  61
 100
  17
   0
  71
  90
  12
  81
  76
  98
  36
  19
  80
  91
  76
  22
  84
  59
  64
  60
  86
  60
  80
  67
  89
  17
  43
  67
  49
  79
  69
  60
  73
  13
  74
  43
  20
  46
   4
  47
  28
  68
  22
  91
  98
  61
   7
  75
  53
  22
   7
  76
  38
  89
  15
  21
  42
  61
  94
  13
  10
  68
  11
  84
 100
  32
  42
  63
  37
  21
  13
  2


Variable containing:
  47
  86
  15
  73
  49
  12
  77
  33
  11
  26
  32
  46
  85
  87
  28
  61
  89
  20
  38
   8
  31
  31
  98
   6
  35
  58
  46
  84
  32
  10
  34
  25
  38
  98
  32
  21
  21
  52
  50
  61
  76
  58
  85
  58
   3
  47
  71
  49
  19
  83
  76
  16
  49
  74
  92
  62
  44
  51
  36
  16
  34
  31
  50
  22
  20
  53
  41
  44
  67
  66
   9
  71
  91
  51
  28
  35
  81
  27
  12
  76
  15
  42
  80
  17
  56
  10
  77
  38
  28
  75
  34
  82
  55
   9
  51
  72
  83
  92
  19
  45
  77
   4
  43
  52
  28
  12
  81
  18
  95
  60
  72
  77
   8
  97
  78
  47
  47
  41
  55
  73
  95
  61
   6
  42
  82
  39
  27
  50
  73
  46
  18
  68
   2
  88
  75
  44
   0
  74
  10
  26
  36
  34
  56
  78
  46
  42
  55
  12
  85
  49
  77
  39
  36
  16
  26
  40
  34
  69
   6
  25
  97
  89
  37
  71
  35
  65
   8
  15
  30
  78
  26
  89
  97
   3
  77
  76
  23
  93
  22
  78
   6
  59
  57
  54
   4
  64
  32
  62
  59
  55
  22
  55
  26
  82
  15
  2

...


Variable containing:
  33
  15
  10
  98
  47
   0
  87
  57
  11
  12
  48
  38
  39
  68
  19
  50
  28
  56
  64
  47
   6
  96
  15
  23
  20
  84
  33
  20
   3
  51
  35
  99
  50
   9
  18
   3
  76
  22
  10
  42
  26
  68
  62
  55
  82
  98
  29
  97
 100
  34
  45
  96
  27
   9
  41
  98
 100
  99
  14
  62
  48
  75
  26
  64
   5
  58
  36
  98
  69
  43
  60
 100
  23
  76
  36
  84
  26
  58
  24
  68
  34
   3
   6
  20
  99
  93
  50
  28
  44
  78
  16
  31
  42
  23
  34
  28
  71
  11
  79
  48
  39
  63
   3
  40
  33
  57
  98
   2
  31
  74
   7
  35
  79
  67
  95
  42
  34
  81
  59
  82
  47
  36
  62
  21
  77
  76
  63
  15
  26
  66
   6
  38
  26
  23
  46
  53
  61
  98
  99
  85
  64
   7
   5
  52
  37
  48
  35
  52
  33
  67
  21
  21
  33
  16
  88
 100
  73
  65
  12
  15
  21
  71
  65
  11
  51
  85
  36
  38
  52
   2
  29
  45
  49
   8
   0
  46
   6
  78
 100
  55
  96
  84
  17
  79
  36
  33
  54
  47
   6
  70
  41
  46
  25
  43
  30
  7

 Variable containing:
1.00000e-02 *
 0.2845 -0.8396 -0.1415  ...  -1.1575  0.3351  1.5769
 0.2845 -0.8396 -0.1415  ...  -1.1575  0.3351  1.5769
 0.2845 -0.8396 -0.1415  ...  -1.1575  0.3351  1.5769
          ...             ⋱             ...          
 0.2845 -0.8396 -0.1415  ...  -1.1575  0.3351  1.5769
 0.2845 -0.8396 -0.1415  ...  -1.1575  0.3351  1.5769
 0.2845 -0.8396 -0.1415  ...  -1.1575  0.3351  1.5769
[torch.cuda.FloatTensor of size 300x101 (GPU 0)]

Variable containing:
  98
  18
  88
   6
  57
  18
  15
  65
  74
  25
  72
  60
  56
  40
  78
  33
  45
  80
  31
  82
  65
   7
  80
  59
  78
  61
  90
  81
  77
  69
  16
  19
  71
  74
  22
  19
  68
  81
   6
   4
  60
  66
  25
  42
  22
  59
  34
  70
  43
  78
  55
  22
  20
  36
 100
  79
  70
  67
  84
  97
   2
  89
  47
  34
  46
  84
  93
  33
  99
  34
  48
  91
  70
  91
  41
  13
  91
  17
  28
   7
  46
  85
  77
  57
  36
  60
  11
  17
  36
   6
  46
   9
  33
  95
  25
  22
  84
  66
  74
  86
  99
   8
  23



Variable containing:
   9
  78
  26
  84
  15
  24
   9
  95
  42
  10
  14
  76
  38
  50
  38
  35
  97
  51
  90
  18
  83
  57
  12
   0
  22
  95
  65
  58
  61
  56
  48
  34
  65
 100
  78
  14
  91
  50
  69
  92
  45
  10
  60
  67
  82
  43
  38
  73
  22
  43
  97
  88
   8
  56
   9
  61
  14
   0
  35
  30
  62
  65
   6
   7
  49
  24
  73
  25
   9
  72
  79
  83
  54
  12
  15
  53
  67
  55
  97
  45
  21
  35
  67
  86
 100
  98
  89
  44
  74
  73
  99
   9
  80
  54
  74
  57
  94
  10
   5
  90
  45
  49
  90
  94
  84
  57
  91
  21
  84
   3
  12
  94
   3
  41
   0
  98
  92
  87
   7
  67
  27
  42
  34
  11
  48
   4
  69
  61
  28
  52
  37
  91
  37
  15
  35
  73
  25
  63
  54
  85
  89
   5
  57
   1
  93
  16
 100
  70
  15
  87
  30
   7
  92
  42
  37
  46
  19
  66
  41
  46
  37
  40
  26
  26
  51
  16
  14
  71
  19
  14
  68
  27
  96
  29
  37
  84
  91
  12
  73
  87
  17
  45
  45
  86
  95
  21
  10
  11
  92
  62
   5
  32
  28
  68
  91
  8

 Variable containing:
1.00000e-02 *
 0.0132 -0.7689 -0.3268  ...  -1.0727  0.2796  1.2319
 0.0132 -0.7689 -0.3268  ...  -1.0727  0.2796  1.2319
 0.0132 -0.7689 -0.3268  ...  -1.0727  0.2796  1.2319
          ...             ⋱             ...          
 0.0132 -0.7689 -0.3268  ...  -1.0727  0.2796  1.2319
 0.0132 -0.7689 -0.3268  ...  -1.0727  0.2796  1.2319
 0.0132 -0.7689 -0.3268  ...  -1.0727  0.2796  1.2319
[torch.cuda.FloatTensor of size 300x101 (GPU 0)]

Variable containing:
  89
  16
  36
  30
   5
  33
  81
  50
  79
  57
  54
  96
  85
  12
   9
 100
  86
 100
  59
  16
  63
  35
  90
  63
 100
   8
  86
  95
  66
  86
  99
  12
  68
  53
  79
  81
  79
  32
  76
  60
  75
  23
  46
  32
  43
  11
  77
  18
  52
  93
  69
  79
  27
  59
  21
  41
  16
  29
  36
  10
  94
  54
  62
  88
   6
  41
  47
  92
  26
  78
  78
  80
  28
  47
  67
  95
  88
  31
  92
  25
  29
  97
  88
  25
  58
  11
  61
  90
  89
  95
  42
  32
  60
  19
  21
  50
  81
  80
  19
  81
   5
  13
  52



Variable containing:
  64
  95
  27
  39
  34
  19
  76
  13
  40
  71
   9
  70
  82
  82
  25
  25
  43
  21
  32
  73
   4
  40
  77
  78
  13
  32
  97
  75
  64
  31
  73
   6
  52
  89
  35
  60
  76
 100
  73
  32
  99
   8
   0
  30
  62
  97
   0
  41
  60
  22
  28
   1
 100
  50
  65
  87
  44
   2
  97
  45
  45
  41
  58
   7
  92
  12
   3
  18
  16
  96
  67
  92
   2
  92
  72
  61
  18
  68
  88
  44
  63
  71
  80
   1
  45
  80
  96
  50
  78
  43
  52
  70
  96
  28
   2
  81
   0
  78
  97
  64
  26
  75
  32
  79
  18
  82
  61
  82
  26
  78
  67
  83
  53
  38
  39
   4
  21
  66
   3
  97
  24
  68
  61
  22
  35
   0
  53
  81
  72
   8
  63
   4
  83
  21
  63
  73
  80
  91
  57
  47
  60
  54
  98
  11
  96
  60
  58
  75
  25
  73
  38
  15
   2
  26
  49
  83
  50
  10
  29
   8
  96
  55
  96
  98
  37
  33
  66
  99
  67
   0
  70
   0
   8
  90
  46
  53
  75
  46
  72
  80
  96
  53
  33
[torch.cuda.LongTensor of size 183 (GPU 0)]
 Variable containin

In [ ]:
# # Test set is not implemented yet

# # Test the Model
# correct = 0
# total = 0
# for images, labels in test_loader:
#     images = Variable(images.view(-1, sequence_length, input_size))
#     outputs = rnn(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum()

# print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 

# # Save the Model
# # torch.save(rnn.state_dict(), 'rnn.pkl')
# # torch.save(rnn, 'rnn.pt')

# Generate dummy data set

In [ ]:
# import numpy as np

In [ ]:
# n_feat = 100
# n_seq = 50 # fixed for now
# n_video = 10

In [ ]:
# for i in range(4,5):
#     tmp_data = np.random.random((n_seq+1, n_feat))
#     np.save('data/features/Archery/v' + str(i) + '.npy', tmp_data)